In [1]:
# 2
import pyspark
from pyspark.sql import SparkSession
# MINIO CONFIGURATION
s3_host = "minio"
s3_url = f"http://{s3_host}:9000"
s3_key = "minio"
s3_secret = "SU2orange!"
s3_bucket = "labd"
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.3.4")\
    .config("spark.hadoop.fs.s3a.endpoint", s3_url) \
    .config("spark.hadoop.fs.s3a.access.key", s3_key) \
    .config("spark.hadoop.fs.s3a.secret.key", s3_secret) \
    .config("spark.hadoop.fs.s3a.fast.upload", True) \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [2]:
logs_in = f"s3a://{s3_bucket}/logs/*.log"
logs_out = f"s3a://{s3_bucket}/logs_no_header"
iplookup_in = f"s3a://{s3_bucket}/iplookup/iplookup.json"
cleanedlogs_out = f"s3a://{s3_bucket}/cleaned-logs.parquet"

print("Stripping Headers...")
#logs2 = logs1.filter(~logs1['value'].startswith("#") ) 
spark.read.text(logs_in)\
    .filter("value not like '#%'")\
    .write.mode("Overwrite").text(logs_out)

Stripping Headers...


Py4JJavaError: An error occurred while calling o44.text.
: java.io.IOException: From option fs.s3a.aws.credentials.provider java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.auth.IAMInstanceCredentialsProvider not found
	at org.apache.hadoop.fs.s3a.S3AUtils.loadAWSProviderClasses(S3AUtils.java:592)
	at org.apache.hadoop.fs.s3a.S3AUtils.createAWSCredentialProviderSet(S3AUtils.java:556)
	at org.apache.hadoop.fs.s3a.DefaultS3ClientFactory.createS3Client(DefaultS3ClientFactory.java:52)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:256)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apach

In [ ]:
df = spark.read.text(logs_in)
#df.select(df.value.substr(0,1),df.value.substr(0,1) != "#" ).show()
df.filter(df.value.substr(0,1) != "#").show()

In [ ]:
print("Loading stripped logs...")
logs = spark.read.option("header",False).option("inferSchema",True).option("sep"," ").csv(logs_out)\
    .toDF("date","time", "serverip", "method", "uri", "querystirng", "port", "username", "clientip", "useragent", "referrer", "statuscode", "a","b","c")\


In [52]:
print("Loading IP lookups...")
ip = spark.read.option("multiline", True).json(iplookup_in)
ip = ip.select("ip",ip.geography.city.alias("city"), "geography.state", "geography.country", "location.lat", "location.lng")

ip.printSchema()


Loading IP lookups...
root
 |-- ip: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)



In [38]:
ip.printSchema()

root
 |-- geography: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- lng: double (nullable = true)



In [47]:
a = ip.select(ip.location.lat, ip.location.lng, ip.geography.state)
b = a.where("geography.state = 'NY'")
b.show(4)

+------------+------------+---------------+
|location.lat|location.lng|geography.state|
+------------+------------+---------------+
|   40.657602|  -73.583184|             NY|
|   43.048122|  -76.147424|             NY|
|   40.712784|  -74.005941|             NY|
|   43.048122|  -76.147424|             NY|
+------------+------------+---------------+



In [20]:
#NO,No,no... dont do this!
spark.read.text(logs_out).toPandas().to_csv("file.csv")

#python generator
#loop that does a yeild

# with open("file","r") as f:
#     for line in f.readlines():
#         print(f)

In [31]:




print("Join DataFrames...") 
comb = ip.join(logs, on = ip.ip == logs.clientip, how="inner")

print("Save as Parquet...")
comb.write.mode("Overwrite").parquet(cleanedlogs_out)

Loading stripped logs...
Loading IP lookups...
Join DataFrames...
Save as Parquet...


In [3]:
#3
logs_in = f"s3a://{s3_bucket}/logs/*.log"
logs1 = spark.read.text(logs_in)
logs1.show()


+--------------------+
|               value|
+--------------------+
|#Software: Micros...|
|       #Version: 1.0|
|#Date: 2016-02-11...|
|#Fields: date tim...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
+--------------------+
only showing top 20 rows



In [8]:
#4 
#logs2 =  logs1.filter("value not like '#%'")
logs2 = logs1.filter(~logs1.value.startswith("#"))
logs2.show()


+--------------------+
|               value|
+--------------------+
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
+--------------------+
only showing top 20 rows



In [10]:
#5 
logs_out = f"s3a://{s3_bucket}/logs_no_header"
logs2.write.text(logs_out)


In [61]:
#6
logs3 = spark.read.option("header",False).option("inferSchema",True).option("sep"," ").csv(logs_out)\
    .toDF("date","time", "serverip", "method", "uri", "querystirng", "port", "username", "clientip", "useragent", "referrer", "statuscode", "a","b","c")\
    .select("date","time", "serverip", "method", "uri", "querystirng", "port", "username", "clientip", "useragent", "referrer", "statuscode")
logs3.show()


+----------+--------+--------------+------+--------------------+-----------+----+--------+-----------+--------------------+--------------------+----------+
|      date|    time|      serverip|method|                 uri|querystirng|port|username|   clientip|           useragent|            referrer|statuscode|
+----------+--------+--------------+------+--------------------+-----------+----+--------+-----------+--------------------+--------------------+----------+
|2016-02-11|17:16:13|128.230.247.37|   GET|                   /|          -|  80|       -|215.82.23.2|Mozilla/5.0+(Wind...|                   -|       200|
|2016-02-11|17:16:13|128.230.247.37|   GET|/Content/jquery-u...|          -|  80|       -|215.82.23.2|Mozilla/5.0+(Wind...|http://group0.ist...|       200|
|2016-02-11|17:16:13|128.230.247.37|   GET|/Plugins/Widgets....|          -|  80|       -|215.82.23.2|Mozilla/5.0+(Wind...|http://group0.ist...|       200|
|2016-02-11|17:16:13|128.230.247.37|   GET|/Plugins/Widgets....|

In [62]:
#7
iplookup_in = f"s3a://{s3_bucket}/iplookup/iplookup.json"
ips1 = spark.read.option("multiline", True).json(iplookup_in)
ips1.show()

+--------------------+---------------+--------------------+
|           geography|             ip|            location|
+--------------------+---------------+--------------------+
|   {Dulles, USA, VA}|  172.189.252.8|{38.955855, -77.4...|
| {Columbus, USA, OH}|    215.82.23.2|{39.961176, -82.9...|
|{Cleveland, USA, OH}|    98.29.25.44|{41.49932, -81.69...|
| {Freeport, USA, NY}|  68.199.40.156|{40.657602, -73.5...|
|{Salt Lake City, ...|155.100.169.152|{40.760779, -111....|
|   {Dallas, USA, TX}|   38.68.15.223|{32.776664, -96.7...|
|    {Tampa, USA, FL}|   70.209.14.54|{27.950575, -82.4...|
|{Arlington, USA, VA}|   74.111.6.173|{38.87997, -77.10...|
| {Syracuse, USA, NY}|128.230.122.180|{43.048122, -76.1...|
| {New York, USA, NY}|128.122.140.238|{40.712784, -74.0...|
|  {Raleigh, USA, NC}| 56.216.127.219|{35.77959, -78.63...|
|{Jersey City, USA...| 54.114.107.209|{40.728157, -74.0...|
| {Syracuse, USA, NY}|   74.111.18.59|{43.048122, -76.1...|
|{Los Angeles, USA...|    8.37.70.170|{3

In [63]:
#8 
ips2 = ips1.select("ip","geography.city", "geography.state", "geography.country", "location.lat", ips1.location.lng.alias("lng") )
ips2.show()

+---------------+--------------+-----+-------+---------+-----------+
|             ip|          city|state|country|      lat|        lng|
+---------------+--------------+-----+-------+---------+-----------+
|  172.189.252.8|        Dulles|   VA|    USA|38.955855| -77.447819|
|    215.82.23.2|      Columbus|   OH|    USA|39.961176| -82.998794|
|    98.29.25.44|     Cleveland|   OH|    USA| 41.49932| -81.694361|
|  68.199.40.156|      Freeport|   NY|    USA|40.657602| -73.583184|
|155.100.169.152|Salt Lake City|   UT|    USA|40.760779|-111.891047|
|   38.68.15.223|        Dallas|   TX|    USA|32.776664| -96.796988|
|   70.209.14.54|         Tampa|   FL|    USA|27.950575| -82.457178|
|   74.111.6.173|     Arlington|   VA|    USA| 38.87997|  -77.10677|
|128.230.122.180|      Syracuse|   NY|    USA|43.048122| -76.147424|
|128.122.140.238|      New York|   NY|    USA|40.712784| -74.005941|
| 56.216.127.219|       Raleigh|   NC|    USA| 35.77959| -78.638179|
| 54.114.107.209|   Jersey City|  

In [68]:
# 9
comb1 = ips2.join(logs3, on = ips2.ip == logs3.clientip, how="inner")
comb1.select("uri").toPandas()


,uri
0,/
1,/Content/jquery-ui-themes/smoothness/jquery-ui...
2,/Plugins/Widgets.NivoSlider/Content/nivoslider...
3,/Plugins/Widgets.NivoSlider/Content/nivoslider...
4,/Scripts/jquery.validate.unobtrusive.min.js
...,...
1117,/Themes/DefaultClean/Content/images/compare-bu...
1118,/Themes/DefaultClean/Content/images/rating1.png
1119,/Themes/DefaultClean/Content/images/rating2.png
1120,/Themes/DefaultClean/Content/images/social-spr...


In [65]:
cleanedlogs_out = f"s3a://{s3_bucket}/cleaned-logs.parquet"
comb1.write.mode("Overwrite").parquet(cleanedlogs_out)